In [1]:
import pandas as pd
import time
import numpy as np

In [12]:
gff3_source = 'c_elegans.PRJNA13758.WS262.annotations.gff3'
genomicFa_source = 'c_elegans.PRJNA13758.WS262.genomic.fa'


In [3]:
def readAnnotations(csv): 
    read = pd.read_csv(csv, header=None, sep='\t',skiprows=8)
    wormbase = read[(read[1]=='WormBase')].reset_index(drop=True)
    wormbase ["alias"] = wormbase [8].map(lambda x: nameFinder(x))
    return wormbase

In [4]:
def proteinCodingAlias (gff3_pd):
    gff3_pd['protein_coding'] = gff3_pd[8].map(lambda x: 'protein_coding' in x)
    gff3_pd = gff3_pd[gff3_pd['protein_coding']==True]
    return gff3_pd.alias.unique()


In [5]:
def nameFinder(row):
    temp = row.replace(',',';').split(';')
    for i in temp:
        if "Transcript" in i:
            return i.split(":")[1]
        elif "sequence_name" in i:
            return i.split("=")[1]
        elif "Alias" in i:
            return i
        

In [6]:
def readGenomicTxt(file):
    with open(file,'r') as fa:
        genom = ''.join([line.rstrip('\n').strip('IVXM') for line in fa])
        return genom.split('>')
    

In [7]:
def RC(seq):
    replacements = {"A": "T", "T": "A","G": "C","C": "G",}
    seq = "".join([replacements.get(c, c) for c in seq])[::-1]
    return seq


In [8]:
def readSources ( gff3_csv, genomicFa_csv):
    t = time.time()
    newbase = readAnnotations(gff3_csv)
    genomicFa = readGenomicTxt(genomicFa_csv)
    uniquePC_Alias = proteinCodingAlias(newbase)
    newbase = newbase[(newbase[2]=='CDS') | (newbase[2]=='three_prime_UTR') | (newbase[2]=='five_prime_UTR')]
    genomicDict = { 'I' : genomicFa[1],
                    'II' : genomicFa[2],
                    'III' : genomicFa[3],
                    'IV' : genomicFa[4],
                    'V' : genomicFa[5],
                    'X' : genomicFa[6],
                    'MtDNA' : genomicFa[7][5:]
                  }
    print( "readSources takes" ,  round(time.time() - t), ' sec')
    return [newbase,genomicFa,uniquePC_Alias,genomicDict]


In [9]:
def metaGenome(gff3Data,uniqueAlias,genomic_Dict):
    t = time.time()
    metaGenome = []
    for i in range(0,len(uniqueAlias)):#len(uniqueAlias)
        alias = uniqueAlias[i]
        aliasBase = gff3Data[gff3Data['alias']==alias]
        seq = ""
        if(len(aliasBase)>1):
            x = aliasBase.T.values
            aliasStart = x[3]
            aliasEnd = x[4]
            aliasChr = x[0][0]
            if(x[6][0]=='+'):
                for i in range(0,len(aliasStart)):
                    seq = seq + genomicDict[aliasChr][aliasStart[i]-1:aliasEnd[i]]
            else:
                for i in reversed(range(len(aliasStart))):
                    seq = seq + RC(genomicDict[aliasChr][aliasStart[i]-1:aliasEnd[i]])
            metaGenome.append(">"+alias+'\n'+ seq) 
    f = open('metaGenome.fa', 'w')
    f.write('\n'.join(metaGenome))
    f.close()
    
    print( "metaGenome takes" ,  round(time.time() - t), ' sec')
    

In [13]:
newbase,genomicFa,uniquePC_Alias,genomicDict = readSources(gff3_source,genomicFa_source)

readSources takes 38  sec


In [ ]:
metaGenome(newbase,uniquePC_Alias,genomicDict)
